In [144]:
import pygame
import random
from pygame.sprite import Group

pygame.init()

# Classe pour representer le jeu
class Game(pygame.sprite.Sprite):

    def __init__(self):
        super().__init__()
        # generer notre joueur
        self.player = Player(self)
        self.all_aliens = pygame.sprite.Group()
        self.pressed = {}

    def start(self):
        self.spawn_alien()
    
    def spawn_alien(self):
        for i in range(12):
            for j in range(5):
                alien = Alien(self)
                alien.rect.x += i*80
                alien.rect.y += j*30
                self.all_aliens.add(alien)

    def check_collision(self, sprite, group):
        return pygame.sprite.spritecollide(sprite, group, False, pygame.sprite.collide_mask)

    def update(self, screen):
        # appliquer l'image du joueur
        screen.blit(self.player.image,self.player.rect)

        self.all_aliens.draw(screen) 

        # deplacer la fusee
        if self.pressed.get(pygame.K_RIGHT) and self.player.rect.x + self.player.rect.width<= screen.get_width():
            self.player.move_right() 
        if self.pressed.get(pygame.K_LEFT) and self.player.rect.x >= -30:
            self.player.move_left()

        # recuperer les projectiles d'un joueur
        for projectile in self.player.all_projectiles:
            projectile.move()

        # appliquer l'image du projectile du joueur
        self.player.all_projectiles.draw(screen)

# Classe pour representer le joueur
class Player(pygame.sprite.Sprite):

    def __init__(self,game):
        super().__init__()
        self.game = game
        self.velocity = 5
        self.image = pygame.image.load("ship.bmp")
        self.rect = self.image.get_rect()
        self.rect.x = 500
        self.rect.y = 500
        self.all_projectiles = pygame.sprite.Group()

    def move_right(self):
        self.rect.x += self.velocity

    def move_left(self):
        self.rect.x -= self.velocity

    def launch_projectile(self):
        # Creer une nouvelle instance de la classe projectile
        if len(self.all_projectiles) == 0:
            self.all_projectiles.add(Projectile(self))

class Projectile(pygame.sprite.Sprite):

    def __init__(self,player):
        super().__init__()
        self.velocity = 10
        self.player = player
        self.image = pygame.image.load("projectile.png")
        self.image = pygame.transform.scale(self.image,(50,50))
        self.rect = self.image.get_rect()
        self.rect.x = player.rect.x
        self.rect.y = player.rect.y

    def remove(self):
        self.player.all_projectiles.remove(self)
        
    def move(self):

        self.rect.y -= self.velocity

        for alien in self.player.game.check_collision(self, self.player.game.all_aliens):
            self.remove()
        
# Classe pour representer les ennemis
class Alien(pygame.sprite.Sprite):

    def __init__(self,game):
        super().__init__()
        self.game = game
        self.image = pygame.image.load("alien.png")
        self.image = pygame.transform.scale(self.image,(30,30))
        self.rect = self.image.get_rect()
        self.rect.x = 20
        self.rect.y = 20
        self.velocity = 3

# main 

screen = pygame.display.set_mode((1080,620))

background = pygame.image.load("bg.jpg")

game = Game()

running = True

while running:

    screen.blit(background,(0,-200))
    
    # appliquer la fenetre de jeu
    game.update(screen)
    

    pygame.display.flip()

    for event in pygame.event.get():    # liste des evenements
        if event.type == pygame.QUIT:   # constante de la fermeture de la fenetre
            running = False
            pygame.quit()

        # detecter si une touche a ete relachee
        elif event.type == pygame.KEYDOWN:
            # quelle touche a ete utilisee
            game.pressed[event.key] = True

            # detecter si la touche espace a ete utilisee pour lancer un projectile
            if event.key == pygame.K_SPACE:
                game.player.launch_projectile()

        elif event.type == pygame.KEYUP:
            # quelle touche a ete relachee
            game.pressed[event.key] = False

    if running:
        game.start()

    